### Prueba 3: usar edad numérica escalada completa del llamante y la categórica de la víctima 

In [1]:
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, classification_report
import gc


In [2]:
dataset_dir = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(''))), 'datasets')

In [3]:
# 1. cargar datasets

llamados_v5= pd.read_excel(os.path.join(dataset_dir, 'xlsx/llamados_v5.xlsx'), parse_dates=['llamado_fecha_hora'])


In [4]:
# 2 reemplazar todos los NS/NC en convive por na

llamados_v5.loc[:, 'victima_convive_agresor'] = llamados_v5['victima_convive_agresor'].replace({'NS/NC': pd.NA})



3. Modificaciones para SVM



In [5]:
# 1. timestamp-fecha_hora encodeado y escalado
scaler = MinMaxScaler()
encoder = OrdinalEncoder()


llamados_v5['timestamp_encoded'] = encoder.fit_transform(llamados_v5[['llamado_fecha_hora']])
llamados_v5['timestamp_encoded_scaled'] = scaler.fit_transform(llamados_v5[['timestamp_encoded']])
    
llamados_v5.timestamp_encoded_scaled



0        0.000000
1        0.000060
2        0.000121
3        0.000181
4        0.000181
           ...   
19138    0.999758
19139    0.999940
19140    0.999819
19141    0.999879
19142    0.999879
Name: timestamp_encoded_scaled, Length: 19143, dtype: float64

In [6]:
#Edad. A edad le hago dos cosas para dos pruebas distintas: 
# 2. escalo los casos completos.
# 3. la paso a categórica, ahí los faltantes quedan codificados como una categoría más, y ahí luego la encodeo como 
# variable categórica con un ordina encoder para preservar el orden. 


# 2.edad escalada para pruebas con casos completos de edad
llamados_v5['victima_edad_escalada'] = scaler.fit_transform(llamados_v5[['victima_edad']])
llamados_v5['llamante_edad_escalada'] = scaler.fit_transform(llamados_v5[['llamante_edad']])


In [7]:
# 3. pasar edad a categórica

def categoria_edad (x):
    if (x >= 0) and (x <= 11) :
        return 'Niñez'
    elif (x >= 12) and (x <=18):
        return 'Adolescencia'
    elif (x >= 19) and (x <=30):
        return 'Juventud'
    elif (x>=31) and (x<=65) :
        return 'Adultez'
    elif x>=66:
        return 'Vejez'
    else:
        return 'NS/NC'



llamados_v5['victima_edad_cat'] = \
llamados_v5.victima_edad.apply(categoria_edad)
llamados_v5['llamante_edad_cat'] = \
llamados_v5.llamante_edad.apply(categoria_edad)


# sobra: victima_edad y llamante_edad  llamado_fecha_hora y timestamp_encoded  


In [8]:

llamados_v5['victima_edad_cat_encoded'] = encoder.fit_transform(llamados_v5[['victima_edad_cat']])

In [9]:
# 4. Variables categóricas que serán encodeadas con ordinal: edad cat (ordinal), momento día (ordinal), 
# estación del año (ordinal).
# Además de encodearlas con ordinal, luego voy a escalar ese encoding ordinal. 
# FUENTE Encoding_Methods_for_Categorical_Data.pdf

# 4.1 pasar edad_cat, momento_dia (V5) y estacion_del_año (V5) a dummy con un encoder ordinal


llamados_v5['llamante_edad_cat_encoded'] = encoder.fit_transform(llamados_v5[['llamante_edad_cat']])


#sobra victima_edad_cat y llamante_edad_cat victima_edad y llamante_edad  llamado_fecha_hora y timestamp_encoded  

In [10]:
llamados_v5['momento_encoded'] = encoder.fit_transform(llamados_v5[['momento_dia']])
llamados_v5['estacion_encoded'] = encoder.fit_transform(llamados_v5[['estacion_del_año']])


#sobra momento_dia y estacion_del_año victima_edad_cat y llamante_edad_cat victima_edad y llamante_edad  llamado_fecha_hora y timestamp_encoded  

In [11]:

# 4.2 escalar las variables ordinales edad, momento del día y estación


llamados_v5['victima_edad_encoded_scaled'] = scaler.fit_transform(llamados_v5[['victima_edad_cat_encoded']])
llamados_v5['llamante_edad_encoded_scaled'] = scaler.fit_transform(llamados_v5[['llamante_edad_cat_encoded']])
llamados_v5['momento_encoded_scaled'] = scaler.fit_transform(llamados_v5[['momento_encoded']])
llamados_v5['estacion_encoded_scaled'] = scaler.fit_transform(llamados_v5[['estacion_encoded']])


#sobra victima_edad_cat_encoded y llamante_edad_cat_encoded, momento_encoded y estacion_encoded
# momento_dia y estacion_del_año victima_edad_cat y llamante_edad_cat victima_edad y llamante_edad  llamado_fecha_hora y timestamp_encoded  


In [12]:
llamados_v5.drop('agresor_fam_no_fam', axis=1, inplace=True)

In [13]:
objetos = {}
for column in llamados_v5.columns:
    if llamados_v5[column].dtypes == object:
        objetos[column] = llamados_v5[column].unique()




In [14]:
# 4.3 Variables de SI/NO encodear con dummy
# Find columns that only contain 'SI' and 'NO'

si_no_columns = []
cat_mas_de_dos = []
for column in llamados_v5.columns:
    if llamados_v5[column].dtypes == object:
        if set(llamados_v5[column].unique()).issubset({'SI', 'NO'}):
            si_no_columns.append(column)
        else:
            cat_mas_de_dos.append(column)
        



In [172]:
cat_mas_de_dos
# sacarle las que ya están encodeadas con ordinal y la target

['llamante_genero',
 'caso_judicializado',
 'victima_genero',
 'victima_nacionalidad',
 'victima_discapacidad',
 'victima_convive_agresor',
 'genero_agresor',
 'agresor_conocido_no_conocido',
 'tipo_vinculo_llamante',
 'hecho_lugar_red',
 'momento_dia',
 'estacion_del_año',
 'llamado_provincia_red',
 'victima_edad_cat',
 'llamante_edad_cat']

In [15]:
encodear_one_hot = ['llamante_genero',
 'caso_judicializado',
 'victima_genero',
 'victima_nacionalidad',
 'victima_discapacidad',
 'genero_agresor',
 'agresor_conocido_no_conocido',
 'tipo_vinculo_llamante',
 'hecho_lugar_red',
 'llamado_provincia_red']

In [16]:
# encodear si/no 

for col in si_no_columns:
    llamados_v5[col] = llamados_v5[col].map({'SI': 1, 'NO':0})

In [17]:
#encodear target
llamados_v5['victima_convive_agresor'] = llamados_v5['victima_convive_agresor'].map({'SI': 1, 'NO':0})

In [18]:
# droppear las columnas que ya sé que no voy a usar
llamados_v5.drop(['victima_edad_cat_encoded', 'llamante_edad_cat_encoded', 
                 'momento_encoded', 'estacion_encoded', 'momento_dia',
                 'estacion_del_año', 'victima_edad_cat', 
                 'llamante_edad_cat', 'victima_edad', 'llamante_edad',  
                 'llamado_fecha_hora', 'timestamp_encoded'], axis=1, inplace=True) 

In [19]:
# encodear con one hot todas las categóricas de más de dos valores: encodear_one_hot

encoder = OneHotEncoder(sparse_output=True) # considerar drop first is sparse false

# Fit and transform the data
encoded_sparse = encoder.fit_transform(llamados_v5[encodear_one_hot])

# Convert the sparse matrix to a dense array
encoded_array = encoded_sparse.toarray()

# Create a DataFrame with the encoded columns
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(encodear_one_hot))

# Drop the original columns that were one-hot encoded
llamados_v5.drop(columns=encodear_one_hot, inplace=True)

# Concatenate the original DataFrame with the encoded DataFrame
llamados_v5 = pd.concat([llamados_v5, encoded_df], axis=1)




In [20]:
# sacar todos los casos vacíos de convive y ponerlos en un nuevo dataset aparte, 
# será el test final a predecir con el mejor modelo

# Create a DataFrame with rows where 'convive' is NA
test_final = llamados_v5[llamados_v5['victima_convive_agresor'].isna()]

# Remove rows where 'convive' is NA from the original DataFrame
llamados_v5 = llamados_v5.dropna(subset=['victima_convive_agresor'])


In [21]:
gc.collect()

112

In [22]:
llamados_v5.isnull().values.any()

True

In [23]:
llamados_v5.drop(['victima_edad_escalada'], axis=1, inplace=True) 

In [24]:
llamados_v5= llamados_v5.dropna(subset=['llamante_edad_escalada'])   

In [25]:
# Separate features and target variable
X = llamados_v5.drop(columns=['victima_convive_agresor'])
y = llamados_v5['victima_convive_agresor']

# Stratified split
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Define the parameter grid for GridSearchCV
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto']
}

# Initialize the SVM model
svm = SVC()

# Perform GridSearchCV to find the best parameters
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)
print("Test set accuracy: ", accuracy_score(y_test, y_pred))
print("Classification report:\n", classification_report(y_test, y_pred))


Best parameters found:  {'C': 1, 'gamma': 'scale', 'kernel': 'poly'}
Best cross-validation accuracy:  0.8803262434943797
Test set accuracy:  0.8919514884233738
Classification report:
               precision    recall  f1-score   support

         0.0       0.91      0.97      0.94      1492
         1.0       0.78      0.55      0.64       322

    accuracy                           0.89      1814
   macro avg       0.84      0.76      0.79      1814
weighted avg       0.89      0.89      0.88      1814



In [ ]:
# Apply the best model to the subset of the original dataset
test_final = test_final.dropna(subset=['llamante_edad_escalada'])
X_na = test_final.drop(columns=['victima_convive_agresor','victima_edad_escalada'])


na_predictions = best_model.predict(X_na)

# Add the predictions to the na_convive_df DataFrame
test_final['victima_convive_agresor_pred'] = na_predictions

print("\nPredictions for NA 'convive' values:")
print(test_final)

In [38]:
print('SI-NO en test',test_final.victima_convive_agresor_pred.value_counts())
print('Tamaño test',test_final.shape)

SI-NO en test victima_convive_agresor_pred
0.0    1350
1.0     146
Name: count, dtype: int64
Tamaño test (1496, 78)


In [45]:
llamados_v5.shape

(9067, 76)

In [47]:
llamados_v5.victima_convive_agresor.value_counts()

victima_convive_agresor
0.0    7457
1.0    1610
Name: count, dtype: int64

In [34]:
dataset_original_proporcion_si = 2756/15085
dataset_original_proporcion_no = 12329/15085
test_final_proporcion_si = 310/4058
test_final_porporcion_no = 3748/4058

In [52]:
test_final.victima_convive_agresor_pred.value_counts()[]

146

In [53]:
print('Proporción de SI en dataset original', llamados_v5.victima_convive_agresor.value_counts()[1]/llamados_v5.shape[0], 'Proporción de SI en test final',test_final.victima_convive_agresor_pred.value_counts()[1]/test_final.shape[0] )
print('Proporción de NO en dataset original', test_final.victima_convive_agresor_pred.value_counts()[1]/test_final.shape[0], 'Proporción de NO en test final', test_final.victima_convive_agresor_pred.value_counts()[0]/test_final.shape[0])


Proporción de SI en dataset original 0.1775670012131907 Proporción de SI en test final 0.09759358288770054
Proporción de NO en dataset original 0.09759358288770054 Proporción de NO en test final 0.9024064171122995


In [ ]:
#Prueba 3: usar edad numérica escalada completa del llamante y la categórica de la víctima
#Prueba 4: usar edad numérica escalada completa de la víctima y la categórica del llamante